In [3]:
! pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [28]:
! pip install --quiet pandas

In [8]:
import getpass
import os
from langchain_openai import AzureChatOpenAI

In [12]:
OPENAI_API_VERSION = ""
AZURE_OPENAI_ENDPOINT = ""
AZURE_OPENAI_API_KEY = ""

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY

model_response_generation = ''
llm = AzureChatOpenAI(openai_api_version="", azure_deployment=model_response_generation, )

In [10]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [68]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

chain = create_sql_query_chain(llm, movies_db)
response = chain.invoke({"question": "Suggest some good movies from de caprio"})
response

'SELECT "movie_title", "imdb_score" FROM movies_details WHERE "actor_1_name" = \'Leonardo DiCaprio\' ORDER BY "imdb_score" DESC LIMIT 5'

In [69]:
db.run(response)

"[('Inception\\xa0', 8.8), ('Django Unchained\\xa0', 8.5), ('The Departed\\xa0', 8.5), ('The Wolf of Wall Street\\xa0', 8.2), ('The Revenant\\xa0', 8.1)]"

In [20]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [22]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

In [23]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are 8 employees.'

In [24]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [25]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice'}`
responded: The tables 'Customer' and 'Invoice' seem to be related to the query as they could contain information about sales and customers' countries. I'll query the schema of these tables.


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	L

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The customers from the USA spent the most with a total sales amount of $523.06.'}

# Movies Database 

In [44]:
import pandas as pd
import sqlite3

In [30]:
movies_df = pd.read_csv("movie_metadata.csv")

In [43]:
# clean up the column names 

movies_df.columns = movies_df.columns.str.strip()

In [45]:
conn = sqlite3.connect('movies.db')

In [46]:
movies_df.to_sql("movies_details", conn, if_exists="replace")
conn.close()

In [70]:
movies_db = SQLDatabase.from_uri("sqlite:///movies.db")
print(movies_db.dialect)
print(movies_db.get_usable_table_names())
movies_db.run("SELECT * FROM movies_details LIMIT 10;")

sqlite
['movies_details']


'[(0, \'Color\', \'James Cameron\', 723.0, 178.0, 0.0, 855.0, \'Joel David Moore\', 1000.0, 760505847.0, \'Action|Adventure|Fantasy|Sci-Fi\', \'CCH Pounder\', \'Avatar\\xa0\', 886204, 4834, \'Wes Studi\', 0.0, \'avatar|future|marine|native|paraplegic\', \'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1\', 3054.0, \'English\', \'USA\', \'PG-13\', 237000000.0, 2009.0, 936.0, 7.9, 1.78, 33000), (1, \'Color\', \'Gore Verbinski\', 302.0, 169.0, 563.0, 1000.0, \'Orlando Bloom\', 40000.0, 309404152.0, \'Action|Adventure|Fantasy\', \'Johnny Depp\', "Pirates of the Caribbean: At World\'s End\\xa0", 471220, 48350, \'Jack Davenport\', 0.0, \'goddess|marriage ceremony|marriage proposal|pirate|singapore\', \'http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1\', 1238.0, \'English\', \'USA\', \'PG-13\', 300000000.0, 2007.0, 5000.0, 7.1, 2.35, 0), (2, \'Color\', \'Sam Mendes\', 602.0, 148.0, 0.0, 161.0, \'Rory Kinnear\', 11000.0, 200074175.0, \'Action|Adventure|Thriller\', \'Christoph Waltz\', 

In [91]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

chain = create_sql_query_chain(llm, movies_db)
response = chain.invoke({"question": "de caprio horror movies"})
response

'SELECT "movie_title", "genres", "director_name" FROM movies_details WHERE "actor_1_name" LIKE \'%de caprio%\' AND "genres" LIKE \'%Horror%\' LIMIT 5'

In [92]:
movies_db.run(response)

''